In [1]:
%pip install exa_py
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv # pip install python-dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
exa_api_key = os.getenv("EXA_API_KEY")

In [3]:
from exa_py import Exa

exa = Exa(exa_api_key)

In [4]:
from openai import OpenAI

openai_client = OpenAI(api_key=openai_api_key)

In [5]:
questions = [
    "How did bats evolve their wings?",
    "How did Rome defend Italy from Hannibal?",
]

In [6]:
highlights_options  = {
    "num_sentences": 7, # how long our highlights should be
    "highlights_per_url": 1, # just get the best highlight for each URL
}

# Let the magic happen!
info_for_llm = []
for question in questions:
    search_response = exa.search_and_contents(question, highlights=highlights_options, num_results=3, use_autoprompt=True)
    info = [sr.highlights[0] for sr in search_response.results]
    info_for_llm.append(info)

In [7]:
info_for_llm

[['Onychonycteris finneyi, the earliest known bat, already possessed powered flight. O. finneyi likely had an undulating flight style that alternated periods of fluttering with gliding. Evidence for this lies in the broad and short nature of O. finneyi wing morphology, which would have made it difficult to efficiently maneuver in the air or sustain flight. Additionally claws were seen on the ends of their forelimb digits (which have since disappeared in modern day bats) giving evidence that O. finneyi was a skilled climber. The common ancestor of all bats is hypothesized to have been an arboreal quadruped of the northern hemisphere. This ancestor is predicted to have lived 64 million years ago at the border of the Cretaceous and Paleogene, based on molecular and paleontological data. There is a gap in the fossil record, and no transitional fossils exist from this quadrupedal ancestor to the appearance of the modern bat.',
  'It is, of course, one of the most notable things about bats: 

In [8]:
responses = []
for question, info in zip(questions, info_for_llm):
  system_prompt = "You are RAG researcher. Read the provided contexts and, if relevant, use them to answer the user's question."
  user_prompt = f"""Sources: {info}
  
  Question: {question}"""

  completion = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt},
    ]
  )
  response = f"""
  Question: {question}
  Answer: {completion.choices[0].message.content}
  """
  responses.append(response)

In [9]:
from pprint import pprint # pretty print
pprint(responses)

['\n'
 '  Question: How did bats evolve their wings?\n'
 '  Answer: Bats evolved their wings from a double membrane stretched over a '
 'framework of finger bones. The wing structure of bats consists of '
 'specialized modifications of their forelimbs, allowing for powered flight. '
 'This adaptation is believed to have evolved from gliding as an escape '
 'mechanism, enabling early bat ancestors to navigate and evade threats in '
 'their arboreal habitats. The evolutionary process of developing wings '
 'enabled bats to efficiently maneuver in the air, allowing them to fly, '
 'change directions quickly, and even apply sudden brakes.\n'
 '  ',
 '\n'
 '  Question: How did Rome defend Italy from Hannibal?\n'
 '  Answer: Rome defended Italy from Hannibal by ruling the seas with their '
 'superior fleet and blocking the coastal route by land, including the route '
 'through the coastal city of Massalia. Hannibal, in response, chose to take '
 'the daring route of moving thousands of miles